In [22]:
# Package importing and dataset loading

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

pd.set_option('display.max_colwidth', None)

# load dataset as a pandas DataFrame
data = pd.read_csv('cleaned_JASSS_dataset.csv', delimiter='|')


# concatenate title, abstract, and keywords
data['concatenated'] = data['Title'] + ' ' + data['Abstract'] + ' ' + data['Keywords']


In [23]:
# Embedding and cosine similarity matrix creation

# selecting the embedding model
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

# compute the embeddings for the concatenated text using the SentenceTransformer model
data['embedding'] = data['concatenated'].apply(lambda x: model.encode(x, convert_to_tensor=True).cpu().numpy())

# compute cosine similarity matrix
embeddings = data['embedding'].tolist()
cosine_sim_matrix = cosine_similarity(embeddings)

In [28]:
# Recommendation

input_paper_index = 1  # change this value to the 'New_ID' of the input paper you want (from 1 - 908)

number_of_recommendations = 5 # choose how many of the top n recommendations will be displayed

input_paper_index  -=1  # v1.1: to account for offset in dataframe indexing

# recommend papers
def recommend_papers(paper_index, cosine_sim_matrix):
    sim_scores = list(enumerate(cosine_sim_matrix[paper_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(number_of_recommendations+1)]  
    paper_indices = [i[0] for i in sim_scores]
    return data.iloc[paper_indices]


# recommend by authors
def recommend_papers_by_authors(paper_index, cosine_sim_matrix, data):
    # extract the authors from the input paper
    input_paper_authors = set(data.loc[paper_index, 'Authors'].split(', '))

    # create an empty dictionary to store the recommendations per author
    recommended_papers_by_authors = {}

    for author in input_paper_authors:
        # filter the papers co-authored by the current author
        authored_papers = data[data['Authors'].apply(lambda x: author in x.split(', '))]

        # compute the similarity scores for the authored papers
        authored_paper_indices = authored_papers.index.tolist()
        sim_scores = list(enumerate(cosine_sim_matrix[paper_index][authored_paper_indices]))

        # sort the similarity scores and keep the top 3 most similar papers, excluding the input paper
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[:4] if sim_scores[0][0] == paper_index else sim_scores[:4]

        # add the recommended papers to the dictionary
        paper_indices = [authored_paper_indices[i[0]] for i in sim_scores if authored_paper_indices[i[0]] != paper_index]
        recommended_papers_by_authors[author] = data.loc[paper_indices]

    return recommended_papers_by_authors


recommended_papers = recommend_papers(input_paper_index, cosine_sim_matrix)

recommended_papers_by_authors_dict = recommend_papers_by_authors(input_paper_index, cosine_sim_matrix, data)



# display the number of recommendations
num_recommendations = len(recommended_papers)
print(f"Number of recommendations: {num_recommendations}\n")

# display input paper with left-aligned text inside the tables
input_paper = data.iloc[[input_paper_index]]

print("Input paper:")
display(input_paper.style.set_properties(**{'text-align': 'left'}))

# display recommended papers with left-aligned text
print("\nRecommended papers:")
display(recommended_papers.style.set_properties(**{'text-align': 'left'}))


# display recommended papers by authors with left-aligned text
print("\nTop 3 similar papers by authors of the input paper:")
for author, papers in recommended_papers_by_authors_dict.items():
    print(f"\nTop 3 similar papers by author {author}:")
    display(papers.style.set_properties(**{'text-align': 'left'}))




Number of recommendations: 5

Input paper:



Recommended papers:



Top 3 similar papers by authors of the input paper:

Top 3 similar papers by author Federico Cecconi:



Top 3 similar papers by author Domenico Parisi:
